#Access Pre-processed data
 Reading the pre-processed athletic_data dataframe

In [1]:
import matplotlib.pyplot as plt
%store -r athletic_data
data = athletic_data

#Removing rows with mismatch in heartrate and power rows length mismatch.
activity_data = data[data.apply(lambda row: len(row['heartrate']) == len(row['power']), axis=1)]

# Flag to keep track of mismatch
mismatch_found = False

for index, row in activity_data.iterrows():
    hr_length = len(row['heartrate'])
    pow_length = len(row['power'])
    if hr_length != pow_length:
        print(f"Mismatch found in row {index+1}: Length of 'hr' column is {hr_length}, Length of 'pow' column is {pow_length}")
        mismatch_found = True

if not mismatch_found:
    print("No mismatch found")

No mismatch found


#STEP 1 : Visualizing the original data by plotting it with pandas

In [ ]:

for index, row in activity_data.iterrows():
    hr_values = row['heartrate']
    pow_values = row['power']
    plt.scatter(pow_values, hr_values)
    # Set x-axis limits
    #plt.xlim(0, max(pow_values))  # Set the lower limit as 0 and the upper limit as the maximum value of x-axis param


plt.xlabel('Power')
plt.ylabel('Heart Rate')
plt.title('ScatterPlot of Heartrate and Power')
plt.legend()
plt.show()


#STEP 2 : Visualizing the heartrate and power data in separate histograms


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Flatten the 'heartrate' and 'power' columns
heartrate_flat = np.concatenate(activity_data['heartrate'].values)
power_flat = np.concatenate(activity_data['power'].values)

# Create subplots for the histograms
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 7))

# Plot the histogram of heartrate
ax1.hist(heartrate_flat, bins='auto')
ax1.set_xlabel('Heartrate')
ax1.set_ylabel('Frequency')
ax1.set_title('Histogram of Heartrate')

# Set x-axis limit for better visibility of data close to zero
ax1.set_ylim(bottom=0)

# Plot the histogram of power
ax2.hist(power_flat, bins='auto')
ax2.set_xlabel('Power')
ax2.set_ylabel('Frequency')
ax2.set_title('Histogram of Power')

# Set x-axis limit for better visibility of data close to zero
ax2.set_ylim(bottom=0)
# Adjust spacing between subplots
plt.tight_layout()

# Display the plot
plt.show()
